#### pytorch搭建神经网络模型，尝试调整模型结构（变更神经元数量，增加隐藏层）来提升模型预测的准确率


In [ ]:
#导入包
import torch
from torchvision.datasets import KMNIST
from torchvision.transforms import ToTensor 
from torch.utils.data import DataLoader
import torch.nn as nn

# 定义超参数
LR = 1e-3
epochs = 100 #训练轮次，100轮左右损失率降到了1以下，1000轮降到0.3左右，Accuracy: 76.03%
BATCH_SIZE = 128  #每批64张


#数据集加载
train_data = KMNIST(root='./kmnist_data', train=True, download=True, transform=ToTensor())
test_data = KMNIST(root='./kmnist_data', train=False, download=True, transform=ToTensor())

#数据分批次处理
train_dl = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True) # shuffle：随机打乱数据


# 模型创建 
# 神经元增加 （训练100轮，神经元256，Accuracy: 60.11%, 神经元1000，Accuracy: 61.39%）=》增加神经元对损失率影响不大
# 增加1个隐藏层（隐藏层2层，训练100轮，神经元256，激活函数选ReLU,Accuracy: 73.64%）sigmoid函数增加隐藏层，损失基本不变
# 隐藏层仅1层，使用ReLU函数，神经元256个，Accuracy: 72.8%
model = nn.Sequential(
    nn.Linear(784, 256),  #输入层
    nn.ReLU(),     #隐藏层
    nn.Linear(256, 128),  # 新增的隐藏层
    nn.ReLU(),
    nn.Linear(128, 10)  #输出层
)


In [30]:
#损失函数和优化器
loss_fn = nn.CrossEntropyLoss() #交叉熵损失函数

#优化器（模型参数更新）
optimizer = torch.optim.SGD(model.parameters(), lr=LR) #SGD优化器

###加上训练轮次
# 训练加速，数据加载器，分批次
for epoch in range(epochs):
    for data, target in train_dl:
        #前向运算
        output = model(data.reshape(-1, 784))
        #计算损失
        loss = loss_fn(output, target) #计算梯度
        #反向传播
        optimizer.zero_grad() #所有参数梯度清零
        loss.backward()  #计算梯度（参数.grad）
        optimizer.step() #更新参数
    print(f'Epoch:{epoch}, Loss:{loss.item()}')

Epoch:0, Loss:2.2180464267730713
Epoch:1, Loss:2.1495659351348877
Epoch:2, Loss:2.0579543113708496
Epoch:3, Loss:1.884940266609192
Epoch:4, Loss:1.771724820137024
Epoch:5, Loss:1.5104624032974243
Epoch:6, Loss:1.4477630853652954
Epoch:7, Loss:1.4398494958877563
Epoch:8, Loss:1.4162606000900269
Epoch:9, Loss:1.3857475519180298
Epoch:10, Loss:1.2009378671646118
Epoch:11, Loss:1.078429102897644
Epoch:12, Loss:1.199627161026001
Epoch:13, Loss:0.9566478729248047
Epoch:14, Loss:0.9635658264160156
Epoch:15, Loss:0.9904351830482483
Epoch:16, Loss:1.065567135810852
Epoch:17, Loss:0.9593588709831238
Epoch:18, Loss:0.796842634677887
Epoch:19, Loss:0.7784572243690491
Epoch:20, Loss:0.8787466883659363
Epoch:21, Loss:0.8522575497627258
Epoch:22, Loss:0.7846777439117432
Epoch:23, Loss:0.9622738361358643
Epoch:24, Loss:0.6354860663414001
Epoch:25, Loss:0.8893347382545471
Epoch:26, Loss:0.8809974193572998
Epoch:27, Loss:0.7500686645507812
Epoch:28, Loss:0.6516140699386597
Epoch:29, Loss:0.6952914595603

In [31]:
# 测试
test_dl = DataLoader(test_data, batch_size=BATCH_SIZE)

correct = 0
total = 0

with torch.no_grad(): #不计算梯度
    for data, target in test_dl:
        output = model(data.reshape(-1, 784))
        _, predicted = torch.max(output, 1) #返回第一个维度的最大值张量
        total += target.size(0) #返回张量的样本量
        correct += (predicted == target).sum().item()
print(f'Accuracy: {correct/total*100}%')

Accuracy: 72.8%
